# 01 — Load and Clean (Bellabeat / Fitbit)

Objective:
- Download Fitbit dataset (Kaggle)
- Load key tables (daily activity, sleep, hourly steps, hourly intensities)
- Standardise column names and datetime fields
- Remove duplicates
- Export cleaned datasets to `data_cleaned/`


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np


In [2]:
import kagglehub

dataset_path = kagglehub.dataset_download("arashnic/fitbit")
print("Path to dataset files:", dataset_path)

dataset_dir = Path(dataset_path)


Path to dataset files: /home/codespace/.cache/kagglehub/datasets/arashnic/fitbit/versions/2


In [3]:
base = dataset_dir / "mturkfitbit_export_4.12.16-5.12.16" / "Fitabase Data 4.12.16-5.12.16"
base.exists(), base


(True,
 PosixPath('/home/codespace/.cache/kagglehub/datasets/arashnic/fitbit/versions/2/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16'))

In [4]:
paths = {
    "daily_activity": base / "dailyActivity_merged.csv",
    "sleep_day": base / "sleepDay_merged.csv",
    "hourly_steps": base / "hourlySteps_merged.csv",
    "hourly_intensities": base / "hourlyIntensities_merged.csv",
}

{key: p.exists() for key, p in paths.items()}


{'daily_activity': True,
 'sleep_day': True,
 'hourly_steps': True,
 'hourly_intensities': True}

In [5]:
daily_activity = pd.read_csv(paths["daily_activity"])
sleep_day = pd.read_csv(paths["sleep_day"])
hourly_steps = pd.read_csv(paths["hourly_steps"])
hourly_intensities = pd.read_csv(paths["hourly_intensities"])

daily_activity.shape, sleep_day.shape, hourly_steps.shape, hourly_intensities.shape


((940, 15), (413, 5), (22099, 3), (22099, 4))

In [6]:
def quick_check(df, name, n=3):
    print(f"\n{name} — shape: {df.shape}")
    display(df.head(n))
    print("Columns:", list(df.columns))

quick_check(daily_activity, "daily_activity")
quick_check(sleep_day, "sleep_day")
quick_check(hourly_steps, "hourly_steps")
quick_check(hourly_intensities, "hourly_intensities")



daily_activity — shape: (940, 15)


,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.74,6.74,0.0,2.44,0.40,3.91,0.0,30,11,181,1218,1776


Columns: ['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance', 'LoggedActivitiesDistance', 'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories']

sleep_day — shape: (413, 5)


,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016 12:00:00 AM,1,327,346
1,1503960366,4/13/2016 12:00:00 AM,2,384,407
2,1503960366,4/15/2016 12:00:00 AM,1,412,442


Columns: ['Id', 'SleepDay', 'TotalSleepRecords', 'TotalMinutesAsleep', 'TotalTimeInBed']

hourly_steps — shape: (22099, 3)


,Id,ActivityHour,StepTotal
0,1503960366,4/12/2016 12:00:00 AM,373
1,1503960366,4/12/2016 1:00:00 AM,160
2,1503960366,4/12/2016 2:00:00 AM,151


Columns: ['Id', 'ActivityHour', 'StepTotal']

hourly_intensities — shape: (22099, 4)


,Id,ActivityHour,TotalIntensity,AverageIntensity
0,1503960366,4/12/2016 12:00:00 AM,20,0.333333
1,1503960366,4/12/2016 1:00:00 AM,8,0.133333
2,1503960366,4/12/2016 2:00:00 AM,7,0.116667


Columns: ['Id', 'ActivityHour', 'TotalIntensity', 'AverageIntensity']


In [7]:
def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"[()]", "", regex=True)
    )
    return df

daily_activity = clean_columns(daily_activity)
sleep_day = clean_columns(sleep_day)
hourly_steps = clean_columns(hourly_steps)
hourly_intensities = clean_columns(hourly_intensities)

daily_activity.columns


Index(['id', 'activitydate', 'totalsteps', 'totaldistance', 'trackerdistance',
       'loggedactivitiesdistance', 'veryactivedistance',
       'moderatelyactivedistance', 'lightactivedistance',
       'sedentaryactivedistance', 'veryactiveminutes', 'fairlyactiveminutes',
       'lightlyactiveminutes', 'sedentaryminutes', 'calories'],
      dtype='object')

In [8]:
# Daily table: ActivityDate -> activitydate -> activity_date
if "activitydate" in daily_activity.columns:
    daily_activity = daily_activity.rename(columns={"activitydate": "activity_date"})

# Sleep table: SleepDay -> sleepday -> sleep_day
if "sleepday" in sleep_day.columns:
    sleep_day = sleep_day.rename(columns={"sleepday": "sleep_day"})

# Hourly tables: ActivityHour -> activityhour -> activity_hour
if "activityhour" in hourly_steps.columns:
    hourly_steps = hourly_steps.rename(columns={"activityhour": "activity_hour"})

if "activityhour" in hourly_intensities.columns:
    hourly_intensities = hourly_intensities.rename(columns={"activityhour": "activity_hour"})

daily_activity.columns, sleep_day.columns, hourly_steps.columns, hourly_intensities.columns


(Index(['id', 'activity_date', 'totalsteps', 'totaldistance', 'trackerdistance',
        'loggedactivitiesdistance', 'veryactivedistance',
        'moderatelyactivedistance', 'lightactivedistance',
        'sedentaryactivedistance', 'veryactiveminutes', 'fairlyactiveminutes',
        'lightlyactiveminutes', 'sedentaryminutes', 'calories'],
       dtype='object'),
 Index(['id', 'sleep_day', 'totalsleeprecords', 'totalminutesasleep',
        'totaltimeinbed'],
       dtype='object'),
 Index(['id', 'activity_hour', 'steptotal'], dtype='object'),
 Index(['id', 'activity_hour', 'totalintensity', 'averageintensity'], dtype='object'))

In [9]:
daily_activity["activity_date"] = pd.to_datetime(daily_activity["activity_date"], errors="coerce")

# Sleep often has a consistent AM/PM format; use explicit format for stability
sleep_day["sleep_day"] = pd.to_datetime(
    sleep_day["sleep_day"],
    format="%m/%d/%Y %I:%M:%S %p",
    errors="coerce"
)

hourly_steps["activity_hour"] = pd.to_datetime(hourly_steps["activity_hour"], errors="coerce")
hourly_intensities["activity_hour"] = pd.to_datetime(hourly_intensities["activity_hour"], errors="coerce")

print("Null date counts:")
print("daily_activity activity_date:", daily_activity["activity_date"].isna().sum())
print("sleep_day sleep_day:", sleep_day["sleep_day"].isna().sum())
print("hourly_steps activity_hour:", hourly_steps["activity_hour"].isna().sum())
print("hourly_intensities activity_hour:", hourly_intensities["activity_hour"].isna().sum())


/tmp/ipykernel_2049/1908996989.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hourly_steps["activity_hour"] = pd.to_datetime(hourly_steps["activity_hour"], errors="coerce")
/tmp/ipykernel_2049/1908996989.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hourly_intensities["activity_hour"] = pd.to_datetime(hourly_intensities["activity_hour"], errors="coerce")


Null date counts:
daily_activity activity_date: 0
sleep_day sleep_day: 0
hourly_steps activity_hour: 0
hourly_intensities activity_hour: 0


In [10]:
def drop_dupes(df, name):
    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    print(f"{name}: removed {before - after} duplicate rows")
    return df

daily_activity = drop_dupes(daily_activity, "daily_activity")
sleep_day = drop_dupes(sleep_day, "sleep_day")
hourly_steps = drop_dupes(hourly_steps, "hourly_steps")
hourly_intensities = drop_dupes(hourly_intensities, "hourly_intensities")


daily_activity: removed 0 duplicate rows
sleep_day: removed 3 duplicate rows
hourly_steps: removed 0 duplicate rows
hourly_intensities: removed 0 duplicate rows


In [11]:
def user_coverage(df, id_col="id"):
    return df[id_col].nunique()

print("Unique users:")
print("daily_activity:", user_coverage(daily_activity))
print("sleep_day:", user_coverage(sleep_day))
print("hourly_steps:", user_coverage(hourly_steps))
print("hourly_intensities:", user_coverage(hourly_intensities))

print("\nDate ranges:")
print("daily_activity:", daily_activity["activity_date"].min(), "to", daily_activity["activity_date"].max())
print("sleep_day:", sleep_day["sleep_day"].min(), "to", sleep_day["sleep_day"].max())
print("hourly_steps:", hourly_steps["activity_hour"].min(), "to", hourly_steps["activity_hour"].max())


Unique users:
daily_activity: 33
sleep_day: 24
hourly_steps: 33
hourly_intensities: 33

Date ranges:
daily_activity: 2016-04-12 00:00:00 to 2016-05-12 00:00:00
sleep_day: 2016-04-12 00:00:00 to 2016-05-12 00:00:00
hourly_steps: 2016-04-12 00:00:00 to 2016-05-12 15:00:00


In [12]:
output_dir = Path("../data_cleaned")
output_dir.mkdir(exist_ok=True)

daily_activity.to_csv(output_dir / "daily_activity_clean.csv", index=False)
sleep_day.to_csv(output_dir / "sleep_day_clean.csv", index=False)
hourly_steps.to_csv(output_dir / "hourly_steps_clean.csv", index=False)
hourly_intensities.to_csv(output_dir / "hourly_intensities_clean.csv", index=False)

sorted([p.name for p in output_dir.glob("*.csv")])


['daily_activity_clean.csv',
 'hourly_intensities_clean.csv',
 'hourly_steps_clean.csv',
 'sleep_day_clean.csv']

## Notebook 01 Complete

Cleaned outputs saved to `data_cleaned/`:
- `daily_activity_clean.csv`
- `sleep_day_clean.csv`
- `hourly_steps_clean.csv`
- `hourly_intensities_clean.csv`
